## python的map、reduce运算

lambda 为匿名函数

In [3]:
arr1 = [1,2,3]
arr2 = map(lambda i:i+1, arr1)
print(list(arr2))

from functools import reduce
print(reduce(lambda i,j:i+j, arr1))

[2, 3, 4]
6


## 模拟 扑克牌游戏

In [32]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])  # 具名函数


class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split( )

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in suits for rank in ranks]
        
    def __len__(self):  # 决定了len()的调用
        return len(self._cards)
    
    def __getitem__(self, position):   # 决定了可以对实例进行索引调用
        return self._cards[position]
    
deck = FrenchDeck()
print(len(deck))
print(deck[9])

52
Card(rank='J', suit='spades')


In [33]:
print(dir(deck)) # dir可以看到类实例的所有方法

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cards', 'ranks', 'suits']


In [34]:
print(deck.__dict__.keys())  #可以通过__dict__查看实例的变量内容
print(len(deck._cards)) #共有52张牌

dict_keys(['_cards'])
52


In [42]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

deck_1 = sorted(deck, key=spades_high) # 获得排序后的卡牌

## 函数的调用

In [49]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due : {:.2f}>'
        return fmt.format(self.total(), self.due())
    
def FidelityPromo(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def BulkItemPromo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount+= item.total() *.1
    return discount

def LargeOrderPromo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellon', 5, 5.0)]
print(Order(joe, cart, LargeOrderPromo))
print(Order(ann, cart, FidelityPromo))

banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]

print(Order(joe, banana_cart, BulkItemPromo))
print(Order(joe, banana_cart, FidelityPromo))

<Order total: 42.00 due : 42.00>
<Order total: 42.00 due : 39.90>
<Order total: 30.00 due : 28.50>
<Order total: 30.00 due : 30.00>


In [50]:
promos = [FidelityPromo, BulkItemPromo, LargeOrderPromo]
def best_promo(order):
    return max(promo(order) for promo in promos)

Order(joe, banana_cart, best_promo)

<Order total: 30.00 due : 28.50>

## 装饰器

In [58]:
def f(func):
    def add(x):
        print(1,x+1)
        return 1
    return add

@f
def use(y):
    return '###'

use(1)

1 2


1

以上逻辑是 f(use(y)) -> add(y) -> print(1,y+1), return 1 -> print(1,2), return 1

和'###'无关